## На рандоме взял страницу с Chanel 

In [2]:
import pandas as pd
import requests
from lxml import html
import xml.etree.ElementTree as ET
from time import sleep
from tqdm import tqdm
import json

In [3]:
#page = requests.get('https://www.fragrantica.ru/designers/Chanel.html')
#data = html.fromstring(page.content)

#### Сайт попался <b>вредный</b>, выдаёт код 429. Я как <s>умный</s> русский человек загуглил и увидел, что это значит слишком много запросов. Поэтому будем обманывать.

In [4]:
url = 'https://www.fragrantica.ru/designers/Chanel.html'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) \
            Chrome/39.0.2171.95 Safari/537.36'}

##### Я знаю, про фэйк юзер эджент, но он почему-то у меня не работает, поэтому пришлось так

In [5]:
page = requests.get(url, headers=headers)
data = html.fromstring(page.content)

In [6]:
page.status_code

200

In [7]:
aromas_list = data.xpath('//div[@class="perfumeslist"]')
gender = data.xpath('//span[@class="mtext"]')

##### Видим, что тут всё довольно однообразно. Берём 1 духи и проверям 

In [8]:
for item in aromas_list[0].text_content().split('\n'):
    if item != '':
        print(item.strip())

Antaeus
1981 для мужчин


In [8]:
for catalog in data.xpath('//div[@class="perfumeslist"]')[0:]:
    print(catalog.text_content().replace('\n',''))

 Antaeus1981 для мужчин
 Bois Noir1987 для мужчин
 Chanel No 461946 для женщин
 Cuir de Russie Parfum1924 для женщин и мужчин
 Gabrielle2017 для женщин
 Le 1940 Beige de Chanel1931 для женщин
 Le 1940 Bleu de Chanel1931 для женщин
 Le 1940 Rouge de Chanel1931 для женщин
 Une Fleur de Chanel1998 для женщин
 Allure1996 для женщин
 Allure eau de parfum1999 для женщин
 Allure Eau Fraichissante Pour l`Ete2002 для женщин
 Allure Hair Mist для женщин
 Allure Homme Eau Fraichissante Pour l`Ete2002 для мужчин
 Allure Homme Edition Blanche2008 для мужчин
 Allure Homme Edition Blanche Eau de Parfum2014 для мужчин
 Allure Homme Sport2004 для мужчин
 Allure Homme Sport Cologne2007 для мужчин
 Allure Homme Sport Eau Extreme2012 для мужчин
 Allure Parfum для женщин
 Allure Pour Homme1999 для мужчин
 Allure Sensuelle2005 для женщин
 Allure Sensuelle Eau de Toilette2006 для женщин
 Allure Sensuelle Parfum2006 для женщин
 Bleu de Chanel2010 для мужчин
 Bleu de Chanel Eau de Parfum2014 для мужчин
 Chance

#####  Получили список духов с этой страницы, с разделением на М/Ж/Унисекс, но пока только в строке 

In [11]:
def getData(data):
    voc = {}
    count = 0 
    for catalog in data.xpath('//div[@class="perfumeslist"]')[0:]:
        for line in catalog.xpath('div/div/p'):
            A = {}
            for a in line.xpath('a'):
                A[a.attrib.get('href')] = a.text_content().replace('\n',',')
            for span in line.xpath('span'):
                A[span.attrib.get('class')] = span.text_content().replace('для',';')
            voc[count] = A
            print(voc[count])
            count += 1
    return pd.DataFrame.from_dict(voc, orient ='index')

#####  Пишем фунцию для забора инфы о духах - название, год и пол. Год и пол лежат в одном, пока хз как разделить 

In [12]:
perfumes = getData(data)

{'/perfume/Chanel/Antaeus-616.html': ' Antaeus', 'mtext': '1981 ; мужчин'}
{'/perfume/Chanel/Bois-Noir-34568.html': ' Bois Noir', 'mtext': '1987 ; мужчин'}
{'/perfume/Chanel/Chanel-No-46-22520.html': ' Chanel No 46', 'mtext': '1946 ; женщин'}
{'/perfume/Chanel/Cuir-de-Russie-Parfum-47794.html': ' Cuir de Russie Parfum', 'mtext': '1924 ; женщин и мужчин'}
{'/perfume/Chanel/Gabrielle-43718.html': ' Gabrielle', 'mtext': '2017 ; женщин'}
{'/perfume/Chanel/Le-1940-Beige-de-Chanel-45188.html': ' Le 1940 Beige de Chanel', 'mtext': '1931 ; женщин'}
{'/perfume/Chanel/Le-1940-Bleu-de-Chanel-45187.html': ' Le 1940 Bleu de Chanel', 'mtext': '1931 ; женщин'}
{'/perfume/Chanel/Le-1940-Rouge-de-Chanel-45186.html': ' Le 1940 Rouge de Chanel', 'mtext': '1931 ; женщин'}
{'/perfume/Chanel/Une-Fleur-de-Chanel-4689.html': ' Une Fleur de Chanel', 'mtext': '1998 ; женщин'}
{'/perfume/Chanel/Allure-502.html': ' Allure', 'mtext': '1996 ; женщин'}
{'/perfume/Chanel/Allure-eau-de-parfum-176.html': ' Allure eau d

#####  И тут заметил, что <b>всё не так просто.</b> Имею в виду, что название духов не выведено в отдельный блок по всем духам. Названию каждый духой соответствует линк на них. Пока хз как это поправить, чтобы вместо отдельного линка у каждых, был бы единый столбец 'name', подобно 'mtext'.

In [13]:
df = pd.DataFrame.from_dict(perfumes)
df.dropna()
df.head()

,/perfume/Chanel/Antaeus-616.html,mtext,/perfume/Chanel/Bois-Noir-34568.html,/perfume/Chanel/Chanel-No-46-22520.html,/perfume/Chanel/Cuir-de-Russie-Parfum-47794.html,/perfume/Chanel/Gabrielle-43718.html,/perfume/Chanel/Le-1940-Beige-de-Chanel-45188.html,/perfume/Chanel/Le-1940-Bleu-de-Chanel-45187.html,/perfume/Chanel/Le-1940-Rouge-de-Chanel-45186.html,/perfume/Chanel/Une-Fleur-de-Chanel-4689.html,...,/perfume/Chanel/Les-Exclusifs-de-Chanel-No-18-7147.html,/perfume/Chanel/Les-Exclusifs-de-Chanel-No-22-7142.html,/perfume/Chanel/Misia-Eau-de-Parfum-41777.html,/perfume/Chanel/No-18-Eau-de-Parfum-41784.html,/perfume/Chanel/No-22-Eau-de-Parfum-41789.html,/perfume/Chanel/Sycomore-4688.html,/perfume/Chanel/Sycomore-Eau-de-Parfum-41780.html,/perfume/Chanel/Pour-Monsieur-615.html,/perfume/Chanel/Pour-Monsieur-Concentree-3131.html,/perfume/Chanel/Pour-Monsieur-Eau-de-Parfum-43001.html
0,Antaeus,1981 ; мужчин,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,1987 ; мужчин,Bois Noir,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,1946 ; женщин,NaN,Chanel No 46,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,1924 ; женщин и мужчин,NaN,NaN,Cuir de Russie Parfum,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,2017 ; женщин,NaN,NaN,NaN,Gabrielle,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### Вполне логично, что пандас <s> умнее меня </s> делает для каждого уникального линка уникальный столбец, тем самым получаем 105 колонок)))))) 